In [1]:
import csv
import pandas as pd
import os
import torch
import MeCab
import ipadic
import json
from tqdm.notebook import tqdm


In [2]:
a = ["a","b"]
b = {k:set() for k in a}
b["a"].add("a")
b

{'a': {'a'}, 'b': set()}

In [ ]:
def view_all_hinshi(text):
    hinshi_set = set()
    CHASEN_ARGS = r' -F "%m\t%f[7]\t%f[6]\t%F-[0,1,2,3]\t%f[4]\t%f[5]\n"'
    CHASEN_ARGS += r' -U "%m\t%m\t%m\t%F-[0,1,2,3]\t\t\n"'
    wakati = MeCab.Tagger(ipadic.MECAB_ARGS + CHASEN_ARGS)

    result = wakati.parse(text).split("\n")
    for line in result:
        wakati_text = line.split("\t")
        if wakati_text[0] != "EOS" and wakati_text[0] != "":
            hinshi_set.add(wakati_text[3])

    return hinshi_set

def extract_hinshi(text,hishi:list):

    hishi_set = {k:set() for k in hishi}
    CHASEN_ARGS = r' -F "%m\t%f[7]\t%f[6]\t%F-[0,1,2,3]\t%f[4]\t%f[5]\n"'
    CHASEN_ARGS += r' -U "%m\t%m\t%m\t%F-[0,1,2,3]\t\t\n"'
    wakati = MeCab.Tagger(ipadic.MECAB_ARGS + CHASEN_ARGS)

    result = wakati.parse(text).split("\n")
    for line in result:
        wakati_text = line.split("\t")
        if wakati_text[0] != "EOS" and wakati_text[0] != "":
            for k in hishi:
                if wakati_text[3] == k:
                    hishi_set[k].add(wakati_text[0])

    return hishi_set


In [4]:
a = {"a":{"a","b"}}
b = {"a":{"ccc","c"}}
a["a"].union(b["a"])

{'a', 'b', 'c', 'ccc'}

In [ ]:
path = "/home/hc/[NII-IDR] 楽天市場データ/review/unzip"
files = os.listdir(path)

first_bar = tqdm(desc="file", total=len(files),position=0)
hss = ["記号-括弧開", "記号-括弧閉","記号-アルファベット","記号-一般","記号-読点","記号-句点"]
all_hinshi = {k:set() for k in hss}

for file in files:

    file_path = os.path.join(path,file)
    col = ['投稿者ID', '店舗名', '店舗ID', '商品名', '商品ID', '商品ページURL', '商品ジャンルID', '商品ジャンルIDパス', '使い道',
                        '目的', '頻度', '評価ポイント', 'レビュータイトル', 'レビュー内容', '参考になった数', 'レビュー登録日時']
    data = pd.read_csv(file_path,sep='\t',quoting=csv.QUOTE_NONE,names=col)
    data.reset_index(inplace=True)
    text = data['レビュー内容']
    second_bar = tqdm(desc="text", total=len(text),position=1)
    for line in text:
        hinshi = extract_hinshi(line,hss)
        count = 0
        for k in hss:
            count += len(all_hinshi[k])
            all_hinshi[k] = all_hinshi[k].union(hinshi[k])
        second_bar.set_postfix(hs=count)
        second_bar.update()


    second_bar.n = 0
    first_bar.update()

with open("hinshi_括号.txt","w") as fp:
    json.dump(all_hinshi,fp)

file:   0%|          | 0/60 [00:00<?, ?it/s]

text:   0%|          | 0/772670 [00:00<?, ?it/s]

text:   0%|          | 0/802581 [00:00<?, ?it/s]

text:   0%|          | 0/786266 [00:00<?, ?it/s]

text:   0%|          | 0/660946 [00:00<?, ?it/s]

...

text:   0%|          | 0/859516 [00:00<?, ?it/s]

text:   0%|          | 0/838356 [00:00<?, ?it/s]

text:   0%|          | 0/823978 [00:00<?, ?it/s]

text:   0%|          | 0/2341440 [00:00<?, ?it/s]

text:   0%|          | 0/1120475 [00:00<?, ?it/s]

In [ ]:
data = pd.read_csv("0.csv")

In [31]:
t = data['レビュー内容'][0]
t

'お祝いに購入。 小さいのに重低音が素晴らしい！想像以上でした。ブラックを購入。ステレオでも聞きたくなり2台目を購入しました。2台目は自分様に。ミッドナイトブルー注文しました。'

In [38]:
view_all_hinshi(t)

{'助動詞',
 '助詞-係助詞',
 '助詞-副助詞',
 '助詞-格助詞-一般',
 '動詞-自立',
 '名詞-サ変接続',
 '名詞-一般',
 '名詞-接尾-一般',
 '名詞-接尾-人名',
 '名詞-接尾-助数詞',
 '名詞-数',
 '名詞-非自立-一般',
 '名詞-非自立-副詞可能',
 '形容詞-自立',
 '接頭詞-名詞接続',
 '記号-一般',
 '記号-句点'}

In [17]:
from bunkai import Bunkai
bunkai = Bunkai()
list(bunkai(t))

['お祝いに購入小さいのに重低音が素晴らしい想像以上でしたブラックを購入ステレオでも聞きたくなり2台目を購入しました2台目は自分様にミッドナイトブルー注文しました']

In [23]:
import MeCab
import ipadic

CHASEN_ARGS = r' -F "%m\t%f[7]\t%f[6]\t%F-[0,1,2,3]\t%f[4]\t%f[5]\n"'
CHASEN_ARGS += r' -U "%m\t%m\t%m\t%F-[0,1,2,3]\t\t\n"'
wakati = MeCab.Tagger(ipadic.MECAB_ARGS + CHASEN_ARGS)



In [24]:
wakati.parse(t)

'お祝い\tオイワイ\tお祝い\t名詞-サ変接続\t\t\nに\tニ\tに\t助詞-格助詞-一般\t\t\n購入\tコウニュウ\t購入\t名詞-サ変接続\t\t\n。\t。\t。\t記号-句点\t\t\n小さい\tチイサイ\t小さい\t形容詞-自立\t形容詞・アウオ段\t基本形\nの\tノ\tの\t名詞-非自立-一般\t\t\nに\tニ\tに\t助詞-格助詞-一般\t\t\n重\tジュウ\t重\t接頭詞-名詞接続\t\t\n低音\tテイオン\t低音\t名詞-一般\t\t\nが\tガ\tが\t助詞-格助詞-一般\t\t\n素晴らしい\tスバラシイ\t素晴らしい\t形容詞-自立\t形容詞・イ段\t基本形\n！\t！\t！\t記号-一般\t\t\n想像\tソウゾウ\t想像\t名詞-サ変接続\t\t\n以上\tイジョウ\t以上\t名詞-非自立-副詞可能\t\t\nでし\tデシ\tです\t助動詞\t特殊・デス\t連用形\nた\tタ\tた\t助動詞\t特殊・タ\t基本形\n。\t。\t。\t記号-句点\t\t\nブラック\tブラック\tブラック\t名詞-一般\t\t\nを\tヲ\tを\t助詞-格助詞-一般\t\t\n購入\tコウニュウ\t購入\t名詞-サ変接続\t\t\n。\t。\t。\t記号-句点\t\t\nステレオ\tステレオ\tステレオ\t名詞-一般\t\t\nでも\tデモ\tでも\t助詞-副助詞\t\t\n聞き\tキキ\t聞く\t動詞-自立\t五段・カ行イ音便\t連用形\nたく\tタク\tたい\t助動詞\t特殊・タイ\t連用テ接続\nなり\tナリ\tなる\t動詞-自立\t五段・ラ行\t連用形\n2\t2\t2\t名詞-数\t\t\n台\tダイ\t台\t名詞-接尾-助数詞\t\t\n目\tメ\t目\t名詞-接尾-一般\t\t\nを\tヲ\tを\t助詞-格助詞-一般\t\t\n購入\tコウニュウ\t購入\t名詞-サ変接続\t\t\nし\tシ\tする\t動詞-自立\tサ変・スル\t連用形\nまし\tマシ\tます\t助動詞\t特殊・マス\t連用形\nた\tタ\tた\t助動詞\t特殊・タ\t基本形\n。\t。\t。\t記号-句点\t\t\n2\t2\t2\t名詞-数\t\t\n台\tダイ\t台\t名詞-接尾-助数詞\t\t\n目\tメ\t目\t名詞-接尾-一般\t\t\nは\tハ